In [1]:
import xml.etree.ElementTree as ET
import requests
import pandas as pd

In [56]:
class XML2DataFrame:
    def __init__(self, xml_data):
        self.root = ET.XML(xml_data)

    def parse_root(self, root):
        return [self.parse_element(child) for child in iter(root)]

    def parse_element(self, element, parsed=None):
        if parsed is None:
            parsed = dict()
        for key in element.keys():
            parsed[key] = element.attrib.get(key)
        if element.text:
            m = {
                'Пол':'sex',
                'Год выпуска':'start_year',
                'Объем':'vol',
                'Тестер':'tester',
                'Семейство':'family',
                'Пробник':'probnik',
            }
            if element.tag == 'param':
                if element.attrib.get('name') == 'Объем':
                    try:
                        ml = ' '+str(element.attrib.get('unit'))
                    except:
                        ml = ''
                    parsed[m[element.attrib.get('name')]] = element.text+ml
                else:
                    parsed[m[element.attrib.get('name')]] = element.text
            else:
                parsed[element.tag] = element.text
        for child in list(element):
            self.parse_element(child, parsed)
        return parsed

    def process_data(self):
        structure_data = self.parse_root(self.root)
        return pd.DataFrame(structure_data)

In [57]:
%%time
xml2df = XML2DataFrame(requests.get('https://pompadoo.ru/catalog/yml/').content)

CPU times: user 720 ms, sys: 94.6 ms, total: 814 ms
Wall time: 12.7 s


In [58]:
#получаем поле offers
xml2df.root = xml2df.root[0][5]

In [59]:
xml_df = xml2df.process_data()

In [61]:
xml_df.head()

,available,categoryId,country_of_origin,cpa,currencyId,delivery,description,family,id,manufacturer_warranty,...,start_year,store,tester,type,typePrefix,unit,url,vendor,vendorCode,vol
0,true,3,Италия,1,RUR,true,NaN,древесные пряные,8138,true,...,2008,false,NaN,vendor.model,Туалетная вода,мл,https://pompadoo.ru/product/4458/?sku=8138,Ermenegildo Zegna,4458,100 мл
1,true,14,Франция,1,RUR,true,NaN,восточные древесные,6162,true,...,2011,false,NaN,vendor.model,Парфюмированная вода,мл,https://pompadoo.ru/product/3488/?sku=6162,Montale,3488,50 мл
2,true,14,Франция,1,RUR,true,NaN,восточные древесные,6161,true,...,2011,false,Да,vendor.model,Парфюмированная вода,мл,https://pompadoo.ru/product/3488/?sku=6161,Montale,3488,100 мл
3,true,14,Франция,1,RUR,true,NaN,восточные древесные,6160,true,...,2011,false,NaN,vendor.model,Парфюмированная вода,мл,https://pompadoo.ru/product/3488/?sku=6160,Montale,3488,100 мл
4,true,11,Франция,1,RUR,true,NaN,древесные ароматические,6905,true,...,2009,false,NaN,vendor.model,Парфюмированная вода,мл,https://pompadoo.ru/product/3501/?sku=6905,Montale,3501,50 мл


In [62]:
xml_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4779 entries, 0 to 4778
Data columns (total 29 columns):
available                4779 non-null object
categoryId               4779 non-null object
country_of_origin        4493 non-null object
cpa                      4779 non-null object
currencyId               4779 non-null object
delivery                 4779 non-null object
description              31 non-null object
family                   4138 non-null object
id                       4779 non-null object
manufacturer_warranty    4779 non-null object
model                    4779 non-null object
name                     4779 non-null object
offer                    4779 non-null object
oldprice                 989 non-null object
pickup                   4779 non-null object
picture                  4779 non-null object
price                    4779 non-null object
probnik                  70 non-null object
sex                      4779 non-null object
start_year              